# Продвинутый Python, лекция 9

**Лектор:** Петров Тимур

**Семинаристы:** Петров Тимур, Коган Александра, Романченко Полина

**Spoiler Alert:** в рамках курса нельзя изучить ни одну из тем от и до досконально (к сожалению, на это требуется больше времени, чем даже 3 часа в неделю). Но мы попробуем рассказать столько, сколько возможно :)

Так как мы проходим веб-разработку, то это явно не та вещь, которая делается через colab. Поэтому здесь написан просто код, который можно воспроизвести локально

## Веб-разработка

Что же, посмотрели, как у других выглядят сайты, что и как они там делают (научились парсить даже все добро), давайте теперь самостоятельно делать! И делать будем, конечно же, с помощью Python

Если вы попробуете вбить в поиск веб-разработка на Python, то увидите, что чаще всего вам предлагают 2 варианта: flask и django.

Ну что же, попробуем освоить эти фреймворки (а также поговорим про Dash к теме визуализации)

А зачем вообще учить оба? Они оба удовлетворяют ваше желание сделать приложение, на разница в том, что:

* Django - очень много решений "из коробки", проще промышленная разработка, но за счет таких решений у вас не так много свободы с точки зрения кастомизации (особенно когда все надо менять часто). То есть Django - это т.н. full stack framework

* Flask - в отличие от Django, это уже light-weight framework. Это значит, что в нем нет решений из коробки, он дает максимальную свободу (и ответственность, ведь приходится самому все реализовывать и делать), но за счет это получается большая кастомизация

Поэтому полезно знать оба фреймворка хотя бы на очень общем уровне (да и в целом, если вам нужно какое-то легкое приложение, то стрелять из пушки по воробьям с помощью Django - идея сомнительная)

## Flask

### Самый простой сайт в мире

Давайте сотворим магию, а потом разберем, что случилось)

In [1]:
%%writefile app.py

from flask import Flask

app = Flask(__name__)

@app.route("/")
def hello_world():
    return "<p>Hello, World!</p>"

if __name__ == '__main__':
    app.run()

Writing app.py


In [ ]:
!flask run

Ура, мы получили самый базовый сайт! Что же мы сделали?

* app - создаем инстанс нашего Web Server Gateway Interface - WSGI (взаимодействие сервера и нашего кода), указываем модуль (если он один, то надо использовать просто __ name __)

* route - декоратор, который

* def hello_world() -> функция, которая будет строить на страничку

* run - запускаем приложение

Запустили код, работает! А теперь хотим изменить код, что делать? Приложение закрыть, исправить, а затем опять запустить. Не нравится, хотим изменять онлайн, ну давайте сделаем!

In [ ]:
app.run(debug=True)

Ура, меняем-обновляем, все работает!

### Усложняем и даем запросы

Теперь хотим не одну страничку, а хотим несколько страниц! Давайте делать

In [ ]:
@app.route('/hello')
def hello():
    return 'Hello World'

Переходим по http://127.0.0.1:5000/hello и получаем необходимый результат

А еще можем делать все динамически! А зачем? А затем, чтобы давать идентификацию (приходишь, а там страничка чисто для тебя)

In [ ]:
@app.route('/user/<username>') # переменные задаются через <>
def show_user_profile(username):
    return 'User %s' % username

@app.route('/post/<int:post_id>') # отдельно можем задать ограничение на тип (например, здесь указываем int)
def show_post(post_id):
    return 'Post %d' % post_id

Ну окей, создали какой-то сайт. А где же те самые запросы POST, GET, о которых мы уже сколько говорили, как там авторизацию делать и всякое такое? Конечно же такое можно задать!

In [ ]:
from flask import request
@app.route('/login', methods=['GET', 'POST']) #указываем методы, которые обрабатываем
def login():
    if request.method == 'POST': # если метод POST, то сделай одно
        do_the_login()
    else:
        show_the_login_form() # иначе другое (GET)

На все ответы с сервера отвечает request. Что тут есть:

* method - какой запрос был (GET, POST etc)

* form - при заполнении формы etc здесь будут находиться переменные 

* args - все наши ключи внутри url-запроса (работает вот так: request.args.get("key", ""))

* files - если были загружены файлы

* cookies - ПЕЧЕНЬКИ!

![](https://i.pinimg.com/originals/08/87/d4/0887d45e7bd3d3aaba12638863df8f48.jpg)

In [ ]:
from flask import request

@app.route('/upload', methods=['GET', 'POST'])
def upload_file():
    if request.method == 'POST':
        f = request.files['the_file'] # для того, чтобы посмотреть на название файла, можно использовать request.filename
        f.save('/var/www/uploads/uploaded_file.txt')

### Генерация шаблонов и статика

Что-то простое разобрали, как просто сделать странички. Но вот есть проблема:

У нас есть сайт, в котором мы хотим иметь единый шаблон (скажем, у нас есть та же вики ФКН, которая на любой странице выглядит одинаково). Прописывать его каждый раз не хочется. Может быть мы можем сделать общий шаблон, по которому все будет генерироваться сразу, а единственное, что нам надо будет делать - это только добавлять новую информацию?

Конечно можем! За шаблоны внутри Flask отвечает библиотека [Jinja2](http://jinja.pocoo.org/docs/templates). Давайте попробуем что-нибудь соорудить уже через шаблонизатор:

In [ ]:
from flask import render_template

@app.route('/hello/')
@app.route('/hello/<name>')
def hello(name=None):
    return render_template('hello.html', name=name)

Опа, словили ошибку! (потому что у нас нет такого шаблона). Надо создать, но где?

Все шаблоны должны находиться в отдельной папке templates. Давайте туда и зафигачим вот такой шаблон:

```
<!doctype html>
<title>Hello from Flask</title>
{% if name %}
    <h1>Hello {{ name }}!</h1>
{% else %}
    <h1>Hello World!</h1>
{% endif %}
```

Ура, успех! Но как это написано немного непонятно, для этого нужно обратиться к [документации](https://jinja.palletsprojects.com/en/3.1.x/templates/)


### Перенаправления

Окей, хотим еще уметь перенаправлять (допустим, у нас есть какая-нибудь страница, с которой мы хотим сразу направить куда-то еще) - скажем, с формы оплаты после оплаты возвращать на начальную страницу

И такое мы умеем!

In [ ]:
from flask import abort, redirect, url_for

@app.route('/')
def index():
    return redirect(url_for('login')) #сделай редирект на страницу с login

@app.route('/login')
def login():
    abort(401) # Выдай ошибку 401   

In [ ]:
from flask import render_template

@app.errorhandler(404) #отдельный декоратор, который будет обрабатывать конкретно ошибку 404
def page_not_found(error):
    return render_template('page_not_found.html'), 404

### Сессии

Чего же не хватает?

Вот у нас есть пользователь, который залогинился и хочет что-нибудь смотреть, но при этом мы хотим сохранять информацию о том, что пользователь залогинен. Как это делать?

Для этого есть объект сессии:

In [ ]:
from flask import Flask, session, redirect, url_for, escape, request
app = Flask(__name__)

@app.route('/')
def index():
    if 'username' in session: 
        return 'Logged in as %s' % escape(session['username']) #escape заменяет все специсимволы на безопасные (потому что можно же ломать сайты) 
    return 'You are not logged in'

@app.route('/login', methods=['GET', 'POST'])
def login():
    if request.method == 'POST':
        session['username'] = request.form['username']
        return redirect(url_for('index'))
    return '''
    <form action="" method="post">
    <p><input type=text name=username>
    <p><input type=submit value=Login> </form>
    '''

@app.route('/logout')
def logout():
    # удалить из сессии имя пользователя, если оно там есть
    session.pop('username', None)
    return redirect(url_for('index'))
    
app.secret_key = 'A0Zr98j/3yX R~XHH!jmN]LWX/,?RT' #секретный ключ (а зачем он? А чтобы пользователь куки не менял)

Как сгенерировать ключ?

In [1]:
import os

os.urandom(24) #подходяший рандомный ключ, максимально случайный

b'@\x11\xaex\xfd5*\x81\xfc\xa3\x9e\xe40\xbc\xff\xb1\xe4\xa1\x99\xa5\xc0VT\x98'

## Применяем наши знания на практике!

Все это здорово, но пока кажется какими-то мазками без общей структуры и смысла, давайте теперь возьмем и это применим, сделав свой минимальный Твиттер.

Давайте ТЗ:

1. Должна быть аутентификация пользователя (имя-логин)

2. Возможность добавить свой собственный текст в ленту (с названием)

3. Наличие ленты, в которой будут все твиты, упорядоченные по дате

Сейчас мы разберемся с самой сложной частью (а именно с работой БД, а далее на семинаре построим уже отображение с точки зрения веба)

### Структура проекта

Первое, что надо сделать - создать структуру. Выглядеть она будет так:

```
/deep_twitter
    deep_twitter.py
    /static - статичные файлы (всякие CSS и JS для отрисовок)
    /templates - шаблоны
```

### Хранение данных

Надо бы где-нибудь данные хранить... А, точно, во Flask так тоже можно сделать! Для этого надо создать схему для таблиц

Создаем файлик ```scheme.sql```, внутри которой напишем:

```
drop table if exists entries;
create table entries (
    id integer primary key autoincrement,
    title text not null,
    text text not null
);
```

Что тут случилось? Мы создали табличку, которая у нас будет хранить все для ленты: id твита, название твита и сам текст

### Разберемся с БД

Напишем несколько функций, которые будут работать с нашим БД

Будем работать с [sqlite3](https://docs.python.org/3/library/sqlite3.html), которая поддерживается Flaskом (на самом деле можно не только sqlite3 использовать, на с ним максимально просто)

In [ ]:
import sqlite3
import os
from flask import Flask, request, redirect, url_for, abort, render_template, flash

def connect_db():
    """Соединяет с указанной базой данных."""
    rv = sqlite3.connect(app.config['DATABASE']) # внутри конфигураций надо будет указать БД, в которую мы будем все хранить
    rv.row_factory = sqlite3.Row #инстанс для итерации по строчкам (может брать по строке и выдавать)
    return rv

def get_db():
    """Если ещё нет соединения с базой данных, открыть новое - для текущего контекста приложения"""
    if not hasattr(g, 'sqlite_db'): #g - это наша глобальная переменная, являющасяс объектом отрисовки
        g.sqlite_db = connect_db()
    return g.sqlite_db

@app.teardown_appcontext #декоратор при разрыве connection
def close_db(error): #закрытие может проходить как нормально, так и с ошибкой, которую можно обрабатывать
    """Закрываем БД при разрыве"""
    if hasattr(g, 'sqlite_db'):
        g.sqlite_db.close()

def init_db():
    """Инициализируем наше БД"""
    with app.app_context(): # внутри app_context app и g связаны
        db = get_db()
    with app.open_resource('schema.sql', mode='r') as f: 
        db.cursor().executescript(f.read())
    db.commit()

Осталось написать функции, которые будут на сайте показывать записи, а также добавлять запись в БД

In [ ]:
@app.route('/')
def show_entries():
    db = get_db()
    cur = db.execute('select title, text from entries order by id desc')
    entries = cur.fetchall()
    return render_template('show_entries.html', entries=entries)

@app.route('/add', methods=['POST'])
def add_entry():
    if not session.get('logged_in'):
        abort(401)
    db = get_db()
    db.execute('insert into entries (title, text) values (?, ?)', [request.form['title'], request.form['text']])
    db.commit()
    flash('New entry was successfully posted')
    return redirect(url_for('show_entries'))

## Животное дня

![](https://animaljournal.ru/articles/wild/primati/koshachiy_lemur/detenish_lemura1.jpg)

Это кошачий лемур. Их все так или иначе видели (по крайней мере вот в таком виде):

![](https://www.meme-arsenal.com/memes/819abc6f23381d803a640e91092ea4a1.jpg)

На Мадагаскаре (где они и обитают) их зовут маки! По размерам как кошка (действительно), при этом хвост может весить примерно половину от всего веса лемура, и это неудивительно - хвост лемура играют важную роль в его жизни.

Помимо самых понятных прикладных функций (с помощью хвоста лемуры удерживают равновесие, будучи на ветках, а также балансируют с его помощью при прыжке), хвост также выполняет социальные функции

С его помощью он более заметен своим сородичам, а также показывают, кто здесь главный (через секрет, которым они этот самый хвост обмазывают)

А еще посмотрите, как они сидят)

![](https://upload.wikimedia.org/wikipedia/commons/thumb/9/93/Ring.tailed.lemur.situp.arp.jpg/1024px-Ring.tailed.lemur.situp.arp.jpg)

Лемуры - социальные животные, живут группой по 30 особей (причем у них матриархат), причем у них максимально яркая социальность: будучи одни, они просто с ума сходят, поэтому нормально изучить их когнитивные способности достаточно сложно